In [1]:
import pandas as pd
from helper_functions.game_states_logic import get_play_states
from helper_functions.pull_game_logic import pull_game
import os
from tqdm import tqdm
from collections import Counter
import numpy as np

In [2]:
team_database = pd.read_excel('Team_Names_Abbrs_v02.xlsx')

In [3]:
existing_transitions = pd.read_excel('ALL_VALID_TRANSITIONS_v02.xlsx')

KeyboardInterrupt: 

In [12]:
files_read = list(existing_transitions['filename'].unique())

In [4]:
for filename in os.listdir('raw_game_pulls'):
    teams = filename.split('for ')[1].split('.xlsx')[0]
    team1, team2 = teams.split(' vs ')
    reduce_df = team_database[team_database['name']==team1].copy()
    if len(reduce_df) > 1:
        print('Collision with {}'.format(team1))
    elif len(reduce_df) == 0:
        print('No match for {}'.format(team1))

    reduce_df = team_database[team_database['name']==team2].copy()
    if len(reduce_df) > 1:
        print('Collision with {}'.format(team2))
    elif len(reduce_df) == 0:
        print('No match for {}'.format(team2))

In [5]:
xs=[]
ys=[]

num_games = 0

In [6]:
#going to re run all with no files read just beacuse some abbreviations have been altered

In [7]:
files_read = []

In [8]:
#RE RUN WITH TIME DIFFERENTIALS LATER

#initialize full dataframe
final_df = pd.DataFrame()

#iterate through files
for filename in tqdm(os.listdir('raw_game_pulls')):
    if filename not in files_read:
        #initialize individual game dataframe
        game_tracking_df = pd.DataFrame()

        #parse filename
        teams = filename.split('for ')[1].split('.xlsx')[0]
        team1, team2 = teams.split(' vs ')
        team_A = team_database[team_database['name']==team1]['abbreviations'].values[0]
        team_B = team_database[team_database['name']==team2]['abbreviations'].values[0]

        #pull file data
        current_data = pd.read_excel('raw_game_pulls/{}'.format(filename))

        #quick check that the abbrev is in there 
        #(had to manually change some because abbrev changed throughout season)
        all_text = ' '.join(list(current_data['PLAY']))
        assert team_A in all_text
        assert team_B in all_text


        #pull states
        all_states = get_play_states(current_data, team_A, team_B)
        final_game = pd.DataFrame()
        for i in range(len(all_states)):
            half_tuples = all_states[i]

            #just take play, not timestamp
            smaller_list = [x[0] for x in half_tuples if x[0] != 'UNNEC']

            #that means rather than this logic, we should count the transitions in here

            # curr_half = pd.DataFrame(smaller_list, columns = ['State','Time'])
            # curr_half['Period']=i+1
            # final_game = pd.concat([final_game,curr_half])

            list_of_pairs = [tuple(smaller_list[j:j+2]) for j in range(len(smaller_list)-2)]

            curr_half = pd.DataFrame({'Transition': list_of_pairs})
            curr_half['Period']=i+1
            game_tracking_df=pd.concat([game_tracking_df,curr_half]).reset_index(drop=True)
        game_tracking_df['filename']=filename
        final_df=pd.concat([final_df,game_tracking_df]).reset_index(drop=True)

  0%|          | 0/4809 [00:00<?, ?it/s]

100%|██████████| 4809/4809 [05:54<00:00, 13.56it/s]


In [9]:
# filename

In [10]:
# team_B

In [11]:
#delete all transitions from FREETHROW state that don't result in either team r or opposite team i
#also delete when freethrow turns to i2 or i3
validate = []
for i in range(len(final_df)):
    curr_tran = final_df['Transition'][i]
    if curr_tran in [('Bi1', 'Af3'),
                       ('Ai2', 'Bf3'),
                       ('Af0', 'Af3'),
                       ('Ai0', 'Bf3'),
                       ('Bf2', 'Bf2'),
                       ('Bi3', 'Af3'),
                       ('Ai0', 'Bf2'),
                       ('Bf3', 'Bf0'),
                       ('Af3', 'Af1'),
                       ('Af3', 'Af0'),
                       ('Bf0', 'Bf2'),
                       ('Bf3', 'Bf1'),
                       ('Ai1', 'Bf3'),
                       ('Br0', 'Af3'),
                       ('Ar0', 'Bf3'),
                       ('Af0', 'Af2'),
                       ('Ai3', 'Bf2'),
                       ('Bi3', 'Af2'),
                       ('Bf2', 'Bf1'),
                       ('Bf2', 'Bf0'),
                       ('Ai2', 'Bf2'),
                       ('Ai1', 'Bf2'),
                       ('Af2', 'Af1'),
                       ('Bi0', 'Af2'),
                       ('Bi2', 'Af2'),
                       ('Af2', 'Af0'),
                       ('Bi1', 'Af2'),
                       ('Bi3', 'Bi3'),
                       ('Ai3', 'Ai3'),
                       ('Ai0', 'Ai3'),
                       ('Bi0', 'Bi3'),
                       ('Ai2', 'Ai3'),
                       ('Ai3', 'Ai2'),
                       ('Ai3', 'Bf0'),
                       ('Br0', 'Af2'),
                       ('Ar0', 'Bf2'),
                       ('Ai1', 'Ai3'),
                       ('Bi3', 'Af0'),
                       ('Bi0', 'Af0'),
                       ('Ai0', 'Ai2'),
                       ('Ai1', 'Bf0'),
                       ('Bi3', 'Bi2'),
                       ('Ai0', 'Bf0'),
                       ('Ai2', 'Ai2'),
                       ('Bi1', 'Bi3'),
                       ('Bi0', 'Bi2'),
                       ('Bi2', 'Bi3'),
                       ('Bi1', 'Af0'),
                       ('Br0', 'Bi3'),
                       ('Ai1', 'Ai2'),
                       ('Bi2', 'Bi2'),
                       ('Bi1', 'Bi2'),
                       ('Ai2', 'Bf0'),
                       ('Ar0', 'Ai3'),
                       ('Bi2', 'Af0'),
                       ('Br0', 'Bi2'),
                       ('Br0', 'Af0'),
                       ('Ar0', 'Ai2'),
                       ('Ar0', 'Bf0'),
                       ('Bi2','Af3'),
                       ('Af3','Af2')]:
        validate.append(False)

    elif ('f' in curr_tran[0])&('f' in curr_tran[1]):
        if (('A' in curr_tran[0])&('A' in curr_tran[1])) | (('B' in curr_tran[0])&('B' in curr_tran[1])):
            validate.append(True)
        else:
            validate.append(False)
    elif (('Af' in curr_tran[0])&('Ai' in curr_tran[1])) | (('Bf' in curr_tran[0])&('Bi' in curr_tran[1])):
        validate.append(False)
    elif ('f' in curr_tran[0])&(('i3' in curr_tran[1]) | ('i2' in curr_tran[1])):
        validate.append(False)
    elif ('1' in curr_tran[1])&('f' not in curr_tran[0]):
        validate.append(False)
    else:
        validate.append(True)

In [12]:
final_df['is_valid']=validate

In [13]:
final_df['is_valid'].value_counts(dropna=False)

is_valid
True     995139
False      6938
Name: count, dtype: int64

In [14]:
final_df[final_df['is_valid']==True]['Transition'].value_counts(dropna=False)[:10]

Transition
(Br0, Ar0)    82705
(Ar0, Br0)    80597
(Ar0, Bi2)    51727
(Br0, Ai2)    42984
(Bi2, Ar0)    37094
(Af0, Af1)    30975
(Ai2, Br0)    29680
(Ar0, Ar0)    29302
(Ar0, Af0)    26829
(Bf0, Bf1)    25987
Name: count, dtype: int64

In [15]:
valid_df = final_df[final_df['is_valid']==True].copy().reset_index(drop=True)

In [23]:
# to_save = pd.concat([existing_transitions,valid_df]).copy().reset_index(drop=True)

In [24]:
# to_save.to_excel('ALL_VALID_TRANSITIONS_v02.xlsx',index=False)

In [16]:
valid_df.to_excel('ALL_VALID_TRANSITIONS_v03.xlsx',index=False)